# System Generation
In this lesson, we will describe the data structures available in PyBigDFT for manipulating the systems we want to study (whether molecular or solid state). This will also be a moment for us to introduce one of the main philosophy's of this framework. In python, we have two very common datastructures: lists and dictionaries.

In [ ]:
my_list = [0, 1, 2, 3]
print(my_list[-1])
my_dict = {"a": "word", "c": 4}
print(my_dict["c"])

These two datastructures have some nice features. First, they are serializable in human readable formats like [json](https://en.wikipedia.org/wiki/JSON) or [yaml](https://en.wikipedia.org/wiki/JSON).

In [ ]:
from yaml import dump
print(dump(my_list))
print(dump(my_dict))

The second is that they can be quickly built and manipulated using comprehensions ([list](https://peps.python.org/pep-0202/) and [dict](https://peps.python.org/pep-0274/)).

In [ ]:
my_list2 = [x * 3 for x in my_list]
print(my_list2)
my_dict2 = {k + "2": v for k, v in my_dict.items()}
print(my_dict2)

## Atom Class
Any system we want to study is going to be made up of Atoms. What is the best way to store information about an atom? From our previous discussion, a `dict` seems most appropriate.

In [ ]:
datm = {"sym": "H", "r": [1, 0, 0], "units": "angstroem"}
print(dump(datm))

Nonetheless, just manipulating a `dict` by itself is error prone, and you might want some helpful subroutines. For this reasons, we've wrapped up a `dict` in our Atom class.

In [ ]:
# let us now install the bigdft client (see previous lessons)
!wget https://gitlab.com/luigigenovese/bigdft-school/-/raw/main/packaging/install.py &> /dev/null
import install
install.client() #such installation is already performed. It should take about 20 sec.

In [ ]:
from BigDFT.Atoms import Atom
atom = Atom(datm)
# the following also works
# atom = Atom({"sym": "H", "r": [1, 0, 0], "units": "angstroem"})
# atom = Atom({"H": [1, 0, 0], "units": "angstroem"})
print(dump(atom))

Some of the built in subroutines are demonstrated below.

In [ ]:
print(atom.sym)
print(atom.atomic_number)
print(atom.get_position("angstroem"))
print(atom.get_position("bohr"))

With this approach, we nonetheless retain the flexibility of a `dict`.

In [ ]:
atom["source"] = "tutorial"
print(atom["source"])
for k, v in atom.items():
    print(k, v)

## Fragments
We won't do many calculations involving a single atom, instead we want to put together groups of atoms. In this case, we will use a list as our model data structure, with the wrapper class refered to as a `Fragment`. 

In [ ]:
atm1 = Atom({"sym": "O", "r": [2.3229430273, 1.3229430273, 1.7139430273], "units": "angstroem"})
atm2 = Atom({"sym": "H", "r": [2.3229430273, 2.0801430273, 1.1274430273], "units": "angstroem"})
atm3 = Atom({"sym": "H", "r": [2.3229430273, 0.5657430273000001, 1.1274430273], "units": "angstroem"})

In [ ]:
from BigDFT.Fragments import Fragment
frag1 = Fragment([atm1, atm2, atm3])
print(len(frag1))
print(frag1.centroid)

It's also possible to build up a fragment in a more step by step process.

In [ ]:
frag1 = Fragment()
frag1.append(atm1)
frag1 += Fragment([atm2])
frag1.extend(Fragment([atm3]))

We added the feature to translate and rotate a fragment.

In [ ]:
from copy import deepcopy
frag2 = deepcopy(frag1)
frag2.translate([10, 0, 0])
frag2.rotate(x=90, units="degrees")

In [ ]:
print(dump(frag2))

## Systems
Many Quantum Mechanical codes top off at the list of atoms level, but in PyBigDFT we go one step further. At the top, we have the `System` class which is based on a `dict`.

In [ ]:
from BigDFT.Systems import System
sys = System()
sys["WAT:0"] = frag1
sys["WAT:1"] = frag2

In [ ]:
print(dump(sys))

In principle, any dictionary key is fine to use for our `System` class, but in practice we follow the convention of giving it a name and identifier separated by a colon. To summarize the hierarchy, let's iterate over our `System`.

In [ ]:
for fragid, frag in sys.items():
    print(fragid)
    for atm in frag:
        print(dict(atm))

Now that we've reached the top level, let's visualize the system we have built.

In [ ]:
# install.packages('py3Dmol') # in case this has not been done before

In [ ]:
_ = sys.display()

The visualization module has identified that there are two separate fragments, and colored them accordingly. Of course if we merged our fragments, the visualization would look different.

In [ ]:
sys2 = System()
sys2["COM:0"] = sum(sys.values())

In [ ]:
_ = sys2.display()

## Shallow Copies and Multiple Views
It is worth recalling the copy semantics of python when dealing with complex objects.

In [ ]:
a = {"x": "1"}
b = {"x": "2"}
my_list = [a, b]
print(my_list)
a["x"] = 3
print(my_list)

We can take advantage of this to construct multiple views of the same molecule. For example, we might want to have two separate views of the same set of atoms. In one view, we've split the set into two molecules, and the other we have just one big fragment. This might be convenient if, for example, we want to be able to rotate the entire system as a group.

In [ ]:
sep = deepcopy(sys)
joint = System()
joint["COM:0"] = sum(sep.values())

In [ ]:
joint["COM:0"].rotate(y=90, units="degrees")

In [ ]:
_ = sep.display()

In [ ]:
_  = joint.display()

## Extended Systems
For extended systems, the extra ingredient required is a `UnitCell`. 

In [ ]:
from BigDFT.UnitCells import UnitCell
sys.cell = UnitCell([7.0, 7.0, 7.0], units="angstroem")

In [ ]:
_ = sys.display()

We see that the minimum image convention has wrapped the red fragment around so that it is now on the left side. We can inspect this position values closer from the `Atom` class.

In [ ]:
for fragid, frag in sys.items():
    print(fragid)
    for at in frag:
        print(at.get_position("angstroem"), at.get_position("angstroem", cell=sys.cell))

We also can get accessed to fractional units this way.

In [ ]:
for fragid, frag in sys.items():
    print(fragid)
    for at in frag:
        print(at.get_position("reduced", cell=sys.cell))

The ```sys.get_posinp()``` method provides the information written in `YAML` markup format:

In [ ]:
sys.get_posinp()

In [ ]:
install.close_drive()